In [3]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
from faraway.datasets import statedata

In [5]:
data = statedata.load()

In [7]:
data.head()

,State,Population,Income,Illiteracy,LifeExp,Murder,HSGrad,Frost,Area
0,AL,3615,3624,2.1,69.05,15.1,41.3,20,50708
1,AK,365,6315,1.5,69.31,11.3,66.7,152,566432
2,AZ,2212,4530,1.8,70.55,7.8,58.1,15,113417
3,AR,2110,3378,1.9,70.66,10.1,39.9,65,51945
4,CA,21198,5114,1.1,71.71,10.3,62.6,20,156361


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   State       50 non-null     object 
 1   Population  50 non-null     int64  
 2   Income      50 non-null     int64  
 3   Illiteracy  50 non-null     float64
 4   LifeExp     50 non-null     float64
 5   Murder      50 non-null     float64
 6   HSGrad      50 non-null     float64
 7   Frost       50 non-null     int64  
 8   Area        50 non-null     int64  
dtypes: float64(4), int64(4), object(1)
memory usage: 3.6+ KB


In [9]:
data.describe()

,Population,Income,Illiteracy,LifeExp,Murder,HSGrad,Frost,Area
count,50.000000,50.000000,50.000000,50.000000,50.00000,50.000000,50.000000,50.000000
mean,4246.420000,4435.800000,1.170000,70.878600,7.37800,53.108000,104.460000,70735.880000
std,4464.491433,614.469939,0.609533,1.342394,3.69154,8.076998,51.980848,85327.299622
min,365.000000,3098.000000,0.500000,67.960000,1.40000,37.800000,0.000000,1049.000000
25%,1079.500000,3992.750000,0.625000,70.117500,4.35000,48.050000,66.250000,36985.250000
50%,2838.500000,4519.000000,0.950000,70.675000,6.85000,53.250000,114.500000,54277.000000
75%,4968.500000,4813.500000,1.575000,71.892500,10.67500,59.150000,139.750000,81162.500000
max,21198.000000,6315.000000,2.800000,73.600000,15.10000,67.300000,188.000000,566432.000000


### Step 2.a Fite the model

In [10]:
mlr_model = smf.ols(formula="Illiteracy ~ Income + Population", data=data).fit()
print(mlr_model.summary())

                            OLS Regression Results                            
Dep. Variable:             Illiteracy   R-squared:                       0.232
Model:                            OLS   Adj. R-squared:                  0.200
Method:                 Least Squares   F-statistic:                     7.110
Date:                Tue, 17 Jun 2025   Prob (F-statistic):            0.00201
Time:                        15:42:36   Log-Likelihood:                -39.081
No. Observations:                  50   AIC:                             84.16
Df Residuals:                      47   BIC:                             89.90
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.1631      0.569      5.556      0.0

Interpretation: Income has a significant negative effect, while Population may or may not be significant based on the p-value.

### Step 2.b Hypothesis Test for Slopes

In [11]:
# Print slopes and p-values
print("Income slope:", mlr_model.params['Income'], "| p-value:", mlr_model.pvalues['Income'])
print("Population slope:", mlr_model.params['Population'], "| p-value:", mlr_model.pvalues['Population'])

Income slope: -0.00047645183275191844 | p-value: 0.0006075011660540417
Population slope: 2.8348404551818606e-05 | p-value: 0.11876474405794932


So, income is statistically significant.

### Step 2c: Confidence Intervals

In [13]:
mlr_model.conf_int(alpha=0.05)

,0,1
Intercept,2.017716,4.308416
Income,-0.000737,-0.000216
Population,-0.000008,0.000064


Confidence intervals for the slopes are shown. If the interval does not include zero, the predictor is significant.

### Step 3 Quadratic model

In [14]:
data['Income2'] = data['Income'] ** 2

# Fit model with quadratic term
quad_model = smf.ols(formula="Illiteracy ~ Income + Income2", data=data).fit()
print(quad_model.summary())

                            OLS Regression Results                            
Dep. Variable:             Illiteracy   R-squared:                       0.399
Model:                            OLS   Adj. R-squared:                  0.373
Method:                 Least Squares   F-statistic:                     15.58
Date:                Tue, 17 Jun 2025   Prob (F-statistic):           6.45e-06
Time:                        15:47:16   Log-Likelihood:                -32.974
No. Observations:                  50   AIC:                             71.95
Df Residuals:                      47   BIC:                             77.68
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.7604      2.452      5.204      0.0

Output Interpretation: The Income2 coefficient indicates curvature. A significant ppp-value confirms nonlinearity.

In [15]:
quad_model.conf_int(alpha=0.05).loc['Income2']

0    2.434836e-07
1    7.292897e-07
Name: Income2, dtype: float64

Output Interpretation: The interval does not contain zero, which implies statistical significance.

### Step 4 Interaction Term

In [17]:
interaction_model = smf.ols(formula="Illiteracy ~ Income * Population", data=data).fit()
print(interaction_model.summary())

                            OLS Regression Results                            
Dep. Variable:             Illiteracy   R-squared:                       0.323
Model:                            OLS   Adj. R-squared:                  0.278
Method:                 Least Squares   F-statistic:                     7.303
Date:                Tue, 17 Jun 2025   Prob (F-statistic):           0.000419
Time:                        15:48:40   Log-Likelihood:                -35.951
No. Observations:                  50   AIC:                             79.90
Df Residuals:                      46   BIC:                             87.55
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             1.7918      0.77

In [18]:
print("Interaction slope:", interaction_model.params['Income:Population'])
print("Confidence Interval:", interaction_model.conf_int().loc['Income:Population'])

Interaction slope: -1.0619842809124954e-07
Confidence Interval: 0   -1.925058e-07
1   -1.989103e-08
Name: Income:Population, dtype: float64


### Step 5 Indicator variable

In [19]:
median_income = data['Income'].median()
data['HighIncome'] = np.where(data['Income'] > median_income, 1, 0)
indicator_model = smf.ols(formula="Illiteracy ~ HighIncome + Population", data=data).fit()
print(indicator_model.summary())

                            OLS Regression Results                            
Dep. Variable:             Illiteracy   R-squared:                       0.111
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     2.943
Date:                Tue, 17 Jun 2025   Prob (F-statistic):             0.0625
Time:                        15:50:32   Log-Likelihood:                -42.739
No. Observations:                  50   AIC:                             91.48
Df Residuals:                      47   BIC:                             97.21
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.2651      0.134      9.444      0.0

### Step 5b & 5c: Hypothesis test and confidence interval for indicator variable

In [20]:
print("HighIncome slope:", indicator_model.params['HighIncome'], "| p-value:", indicator_model.pvalues['HighIncome'])
print("Confidence Interval:", indicator_model.conf_int().loc['HighIncome'])

HighIncome slope: -0.38872592739723566 | p-value: 0.026151809876558442
Confidence Interval: 0   -0.729254
1   -0.048198
Name: HighIncome, dtype: float64


### Step 6a: Log Transformation for Population

In [21]:
data['log_Population'] = np.log(data['Population'])

# Fit model with transformed variable
log_model = smf.ols(formula="Illiteracy ~ Income + log_Population", data=data).fit()
print(log_model.summary())

                            OLS Regression Results                            
Dep. Variable:             Illiteracy   R-squared:                       0.245
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     7.610
Date:                Tue, 17 Jun 2025   Prob (F-statistic):            0.00137
Time:                        15:58:21   Log-Likelihood:                -38.676
No. Observations:                  50   AIC:                             83.35
Df Residuals:                      47   BIC:                             89.09
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          2.0630      0.797      2.

### Step 6b & 6c: Hypothesis test and confidence interval for log transformation

In [22]:
print("log_Population slope:", log_model.params['log_Population'], "| p-value:", log_model.pvalues['log_Population'])
print("Confidence Interval:", log_model.conf_int().loc['log_Population'])

log_Population slope: 0.13554273816057266 | p-value: 0.07423924362419751
Confidence Interval: 0   -0.013805
1    0.284890
Name: log_Population, dtype: float64


### Step 7 Comparisson

In [23]:
models = {
   "Multiple Regression": mlr_model.rsquared_adj,
   "Quadratic Model": quad_model.rsquared_adj,
   "Interaction Model": interaction_model.rsquared_adj,
   "Indicator Model": indicator_model.rsquared_adj,
   "Log Model": log_model.rsquared_adj
}


# Display models in descending order of Adjusted R-squared
import pandas as pd
comparison_df = pd.DataFrame(models.items(), columns=['Model', 'Adjusted R-squared']).sort_values(by='Adjusted R-squared', ascending=False)
print(comparison_df)

                 Model  Adjusted R-squared
1      Quadratic Model            0.373065
2    Interaction Model            0.278437
4            Log Model            0.212467
0  Multiple Regression            0.199609
3      Indicator Model            0.073486
